In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

from libs.common import *
from config.config import *

c:\Users\Admin\Data\ads_fraud_detection


c:/Users/Admin/Data/ads_fraud_detection


# 3. Tiền xử lí dữ liệu

In [ ]:
save_dir=f"{exps_dir}/exp2/exp_gancopula"

save_dir = f"{save_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [3]:
test_size=0.33
seed=42
train=pd.read_csv(f'{exps_dir}/exp2/train_EDA.csv')
test=pd.read_csv(f'{exps_dir}/exp2/test_EDA.csv')
train

,EVENT_ID,DISBURSED_AMOUNT,ASSET_COST,LTV,EMPLOYMENT_TYPE,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,PASSPORT_FLAG,PERFORM_CNS_SCORE,PERFORM_CNS_SCORE_DESCRIPTION,PRI_NO_OF_ACCTS,PRI_ACTIVE_ACCTS,PRI_OVERDUE_ACCTS,PRI_CURRENT_BALANCE,PRI_SANCTIONED_AMOUNT,PRI_DISBURSED_AMOUNT,SEC_NO_OF_ACCTS,SEC_ACTIVE_ACCTS,SEC_OVERDUE_ACCTS,SEC_CURRENT_BALANCE,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,AVERAGE_ACCT_AGE,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,LOAN_DEFAULT,UNIQUEID,EVENT_TIMESTAMP,ENTITY_TYPE,DISBURSED_AMOUNT_new,DISBURSED_AMOUNT_bins,ASSET_COST_new,ASSET_COST_bins,LTV_new,LTV_bins,PERFORM_CNS_SCORE_bins,PRI_NO_OF_ACCTS_new,PRI_NO_OF_ACCTS_bins,PRI_OVERDUE_ACCTS_new,PRI_OVERDUE_ACCTS_bins,age
0,462711,33484,62644,55.23,Salaried,1,1,0,0,0,0,743,C-Very Low Risk,9,5,0,160423,230489,194538,0,0,0,0,0,0,9149,0,4,0,0.583333,1.333333,1,0,f1f85578-53dc-4144-a1e7-004d17cefa6c,2024-11-08T03:42:01Z,user,33484.0,Low,62644.000000,Low,54340.127169,Low,High,54340.127169,More than One,54340.127169,NaN,12310
1,558674,66882,81187,84.37,Salaried,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,f0efe48d-065f-4791-b954-51d5d87b2bb4,2024-04-26T01:19:57Z,user,66882.0,Extreme,81187.000000,Extreme,54340.127169,Extreme,No History,54340.127169,One,54340.127169,NaN,11123
2,528251,59113,71757,84.87,Self employed,1,1,0,0,0,0,738,C-Very Low Risk,3,3,0,45828,58582,58582,0,0,0,0,0,0,4240,0,3,0,0.166667,0.333333,0,1,f17c7d6a-2ae4-49d3-972f-0021ec1925eb,2024-07-15T01:29:01Z,user,59113.0,High,71757.000000,High,54340.127169,Extreme,High,54340.127169,More than One,54340.127169,NaN,11015
3,633950,56059,71307,81.34,Salaried,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,1,d0b18bcb-cede-45fb-9f13-e9186fefea8d,2024-07-28T10:34:54Z,user,56059.0,High,71307.000000,High,54340.127169,High,No History,54340.127169,One,54340.127169,NaN,19781
4,476747,56759,67100,85.69,Self employed,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,b14a79a5-32c4-4cb6-9a1a-6fb7c8eff99b,2024-03-31T10:35:20Z,user,56759.0,High,67100.000000,Medium,54340.127169,Extreme,No History,54340.127169,One,54340.127169,NaN,17984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186518,643379,55549,82230,70.78,Self employed,1,0,0,1,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,ba2758df-48e1-4271-a17e-b348f6c7272f,2024-08-10T14:57:35Z,user,55549.0,High,82230.000000,Extreme,54340.127169,Medium,No History,54340.127169,One,54340.127169,NaN,12111
186519,587710,60213,71857,84.89,Salaried,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,9d25599e-8eaa-49a6-9c78-80cd07056d0e,2024-11-09T10:31:25Z,user,60213.0,High,71857.000000,High,54340.127169,Extreme,No History,54340.127169,One,54340.127169,NaN,16128
186520,636081,52328,68757,78.54,Self employed,1,1,0,0,1,0,300,M-Very High Risk,2,2,1,107853,177000,177000,0,0,0,0,0,0,1732,0,0,0,1.416667,1.750000,0,0,a6bc5e05-6c1b-45ae-8bd6-1850b20e4fb9,2024-10-01T13:12:28Z,user,52328.0,Medium,68757.000000,Medium,54340.127169,High,Low,54340.127169,More than One,54340.127169,One (or more),11200
186521,431351,77579,100360,79.71,Salaried,1,1,0,0,0,0,407,K-High Risk,20,10,6,4712581,5277128,5418148,0,0,0,0,0,0,49484,0,0,6,4.416667,15.416667,1,0,0890f8d4-d88b-4e69-b2bd-28a904491196,2024-12-28T13:04:21Z,user,77579.0,Extreme,54340.127169,Extreme,54340.127169,High,Medium,54340.127169,More than One,54340.127169,One (or more),21460


In [5]:
train_con = train[['EMPLOYMENT_TYPE', 'MOBILENO_AVL_FLAG', 'AADHAR_FLAG', 'PAN_FLAG', 'VOTERID_FLAG',
                   'DRIVING_FLAG', 'PASSPORT_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 
                   'PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 
                   'SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT', 
                   'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 'NO_OF_INQUIRIES',
                   'LOAN_DEFAULT', 'DISBURSED_AMOUNT_new', 'ASSET_COST_new', 
                   'LTV_new', 'age']]

In [6]:
train_bin = train [['UNIQUEID', 'EMPLOYMENT_TYPE', 'MOBILENO_AVL_FLAG', 'AADHAR_FLAG', 'PAN_FLAG', 
                    'VOTERID_FLAG', 'DRIVING_FLAG', 'PASSPORT_FLAG', 'PERFORM_CNS_SCORE', 
                    'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE',
                    'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE',
                    'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT', 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 
                    'CREDIT_HISTORY_LENGTH', 'NO_OF_INQUIRIES', 'LOAN_DEFAULT',
                    'DISBURSED_AMOUNT_bins', 'ASSET_COST_bins', 'LTV_bins',
                    'PERFORM_CNS_SCORE_bins', 'PRI_NO_OF_ACCTS_bins', 'PRI_OVERDUE_ACCTS_bins', 'age']]

In [7]:
test_con = test[['EMPLOYMENT_TYPE', 'MOBILENO_AVL_FLAG', 'AADHAR_FLAG', 'PAN_FLAG', 'VOTERID_FLAG',
                   'DRIVING_FLAG', 'PASSPORT_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 
                   'PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 
                   'SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT', 
                   'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 'NO_OF_INQUIRIES',
                   'LOAN_DEFAULT', 'DISBURSED_AMOUNT_new', 'ASSET_COST_new', 
                   'LTV_new', 'age']]

In [8]:
# Confusion Matrix

def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title = 'Confusion matrix"',
                          cmap = plt.cm.Blues) :
    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])) :
        plt.text(j, i, cm[i, j],
                 horizontalalignment = 'center',
                 color = 'white' if cm[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [9]:
scaler_data = StandardScaler() 
def scaleColumns(df, cols_to_scale):

    for col in cols_to_scale:

        df[col] = pd.DataFrame(scaler_data.fit_transform(pd.DataFrame(train_con[col])),columns=[col])

    return df

In [11]:
scaled_df = scaleColumns(train_con,['PERFORM_CNS_SCORE','PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 
                                 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_OVERDUE_ACCTS', 
                                 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT',
                                 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 
                                 'NO_OF_INQUIRIES', 'DISBURSED_AMOUNT_new',
                                 'ASSET_COST_new', 'LTV_new', 'age'])

scaled_df.head()

,EMPLOYMENT_TYPE,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,PASSPORT_FLAG,PERFORM_CNS_SCORE,PERFORM_CNS_SCORE_DESCRIPTION,PRI_ACTIVE_ACCTS,PRI_CURRENT_BALANCE,PRI_DISBURSED_AMOUNT,SEC_NO_OF_ACCTS,SEC_OVERDUE_ACCTS,SEC_CURRENT_BALANCE,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,LOAN_DEFAULT,DISBURSED_AMOUNT_new,ASSET_COST_new,LTV_new,age
0,Salaried,1,1,0,0,0,0,1.340292,C-Very Low Risk,2.031893,-0.005417,-0.016197,-0.093996,-0.066024,-0.032864,-0.025404,-0.020499,-0.252023,-0.009193,1.127407,0,-1.810134,-0.561253,0.0,-0.734524
1,Salaried,1,1,0,0,0,0,-0.855060,No Bureau History Available,-0.533268,-0.178010,-0.181889,-0.093996,-0.066024,-0.032864,-0.087314,-0.020499,-0.252023,-0.568043,-0.291316,0,1.178058,1.368304,0.0,-1.065269
2,Self employed,1,1,0,0,0,0,1.325519,C-Very Low Risk,1.005829,-0.128706,-0.131993,-0.093996,-0.066024,-0.032864,-0.058623,-0.020499,-0.252023,-0.428330,-0.291316,1,0.482948,0.387032,0.0,-1.095362
3,Salaried,1,1,0,0,0,0,-0.855060,No Bureau History Available,-0.533268,-0.178010,-0.181889,-0.093996,-0.066024,-0.032864,-0.087314,-0.020499,-0.252023,-0.568043,-0.291316,1,0.209700,0.340206,0.0,1.347191
4,Self employed,1,1,0,0,0,0,-0.855060,No Bureau History Available,-0.533268,-0.178010,-0.181889,-0.093996,-0.066024,-0.032864,-0.087314,-0.020499,-0.252023,-0.568043,-0.291316,0,0.272331,-0.097568,0.0,0.846476


In [12]:
train_dummy = pd.get_dummies(scaled_df, prefix_sep='_', drop_first=True)
train_dummy.head()

,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,PASSPORT_FLAG,PERFORM_CNS_SCORE,PRI_ACTIVE_ACCTS,PRI_CURRENT_BALANCE,PRI_DISBURSED_AMOUNT,SEC_NO_OF_ACCTS,SEC_OVERDUE_ACCTS,SEC_CURRENT_BALANCE,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,LOAN_DEFAULT,DISBURSED_AMOUNT_new,ASSET_COST_new,LTV_new,age,EMPLOYMENT_TYPE_Salaried,EMPLOYMENT_TYPE_Self employed,PERFORM_CNS_SCORE_DESCRIPTION_B-Very Low Risk,PERFORM_CNS_SCORE_DESCRIPTION_C-Very Low Risk,PERFORM_CNS_SCORE_DESCRIPTION_D-Very Low Risk,PERFORM_CNS_SCORE_DESCRIPTION_E-Low Risk,PERFORM_CNS_SCORE_DESCRIPTION_F-Low Risk,PERFORM_CNS_SCORE_DESCRIPTION_G-Low Risk,PERFORM_CNS_SCORE_DESCRIPTION_H-Medium Risk,PERFORM_CNS_SCORE_DESCRIPTION_I-Medium Risk,PERFORM_CNS_SCORE_DESCRIPTION_J-High Risk,PERFORM_CNS_SCORE_DESCRIPTION_K-High Risk,PERFORM_CNS_SCORE_DESCRIPTION_L-Very High Risk,PERFORM_CNS_SCORE_DESCRIPTION_M-Very High Risk,PERFORM_CNS_SCORE_DESCRIPTION_No Bureau History Available,PERFORM_CNS_SCORE_DESCRIPTION_Not Scored: More than 50 active Accounts found,PERFORM_CNS_SCORE_DESCRIPTION_Not Scored: No Activity seen on the customer (Inactive),PERFORM_CNS_SCORE_DESCRIPTION_Not Scored: No Updates available in last 36 months,PERFORM_CNS_SCORE_DESCRIPTION_Not Scored: Not Enough Info available on the customer,PERFORM_CNS_SCORE_DESCRIPTION_Not Scored: Only a Guarantor,PERFORM_CNS_SCORE_DESCRIPTION_Not Scored: Sufficient History Not Available
0,1,1,0,0,0,0,1.340292,2.031893,-0.005417,-0.016197,-0.093996,-0.066024,-0.032864,-0.025404,-0.020499,-0.252023,-0.009193,1.127407,0,-1.810134,-0.561253,0.0,-0.734524,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1,1,0,0,0,0,-0.855060,-0.533268,-0.178010,-0.181889,-0.093996,-0.066024,-0.032864,-0.087314,-0.020499,-0.252023,-0.568043,-0.291316,0,1.178058,1.368304,0.0,-1.065269,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
2,1,1,0,0,0,0,1.325519,1.005829,-0.128706,-0.131993,-0.093996,-0.066024,-0.032864,-0.058623,-0.020499,-0.252023,-0.428330,-0.291316,1,0.482948,0.387032,0.0,-1.095362,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,1,1,0,0,0,0,-0.855060,-0.533268,-0.178010,-0.181889,-0.093996,-0.066024,-0.032864,-0.087314,-0.020499,-0.252023,-0.568043,-0.291316,1,0.209700,0.340206,0.0,1.347191,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
4,1,1,0,0,0,0,-0.855060,-0.533268,-0.178010,-0.181889,-0.093996,-0.066024,-0.032864,-0.087314,-0.020499,-0.252023,-0.568043,-0.291316,0,0.272331,-0.097568,0.0,0.846476,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False


In [ ]:
y_train = train_dummy[['LOAN_DEFAULT']]
x_train= train_dummy.loc[:, train_dummy.columns != 'LOAN_DEFAULT']
x_train.shape

(186523, 43)

In [ ]:
x_train = x_train.fillna(0)
x_train.shape

(186523, 43)

In [18]:
y_train

,LOAN_DEFAULT
87780,0
20197,0
79514,0
57866,0
30238,0
...,...
151535,0
55293,0
49751,1
136767,1


In [19]:
y_train.values

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [0]], dtype=int64)

In [20]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train.LOAN_DEFAULT), y=y_train.LOAN_DEFAULT)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict

{0: 0.6370086404152864, 1: 2.3247024365925095}

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_rosampled, y_rosampled = ros.fit_resample(train.drop(columns=['LOAN_DEFAULT']), train['LOAN_DEFAULT'])
X_rosampled = pd.DataFrame(X_rosampled)
y_rosampled = pd.DataFrame(y_rosampled)
oversampling_sample=pd.concat([X_rosampled, y_rosampled], axis=1)    

In [ ]:
from sdv.metadata import SingleTableMetadata
metadata_sync = SingleTableMetadata()
metadata_sync.detect_from_dataframe(data=oversampling_sample)

In [ ]:
from sdv.single_table import CopulaGANSynthesizer


gan_synthesizer = CopulaGANSynthesizer(metadata_sync,
                                            enforce_min_max_values=True,  
                                            enforce_rounding=True,
                                            default_distribution='gamma') 

gan_synthesizer.fit(oversampling_sample)
gan_synthesizer.save(
    filepath=f'{save_dir}/gan_copula_synthesizer.pkl'
)

In [ ]:
gan_synthetic_data = gan_synthesizer.sample(num_rows=len(train))
gan_synthetic_data.head()

In [ ]:
from sdv.evaluation.single_table import run_diagnostic



diagnostic_gau = run_diagnostic(
    real_data=oversampling_sample,
    synthetic_data=gan_synthetic_data,
    metadata=metadata_sync
)




In [ ]:
from sdv.evaluation.single_table import evaluate_quality



diagnostic_gau = evaluate_quality(
    real_data=oversampling_sample,
    synthetic_data=gan_synthetic_data,
    metadata=metadata_sync
)




In [ ]:
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

## gan data
gan_data=gan_synthetic_data[gan_synthetic_data.LOAN_DEFAULT==1]
gan_sample=pd.concat([train, gan_data] , axis=0, ignore_index=True)
X_gansampled, y_gansampled = rus.fit_resample(gan_sample.drop(columns=['LOAN_DEFAULT']), gan_sample['LOAN_DEFAULT'])
X_gansampled = pd.DataFrame(X_gansampled)
y_gansampled = pd.DataFrame(y_gansampled)
gan_sample=pd.concat([X_gansampled, y_gansampled], axis=1)    
gan_sample = shuffle(gan_sample, random_state=42)
print(gan_sample.LOAN_DEFAULT.value_counts())

In [ ]:
gan_sample.to_csv(f'{save_dir}/data_train.csv', index=False)

In [ ]:
x_train = gan_sample.drop(columns=['LOAN_DEFAULT'])
y_train = gan_sample['LOAN_DEFAULT']

In [ ]:
from sklearn.model_selection import train_test_split
x_train.to_csv(f'{save_dir}/x_train.csv', index=False)
y_train.to_csv(f'{save_dir}/y_train.csv', index=False)


In [22]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
print(f"+ X_train: {len(x_train)}")
for fold, (train_idx, valid_idx) in enumerate(kfold.split(x_train, y_train)):
    print(f'Fold {fold}: ')
    print(f'+ train_idx: {train_idx}')
    print(f'+ valid_idx: {valid_idx}')
    print(f'+ train / valid: {valid_idx}')
    pass

+ X_train: 149218
Fold 0: 
+ train_idx: [     1      2      3 ... 149213 149214 149216]
+ valid_idx: [     0      4     12 ... 149212 149215 149217]
+ train / valid: [     0      4     12 ... 149212 149215 149217]
Fold 1: 
+ train_idx: [     0      1      2 ... 149215 149216 149217]
+ valid_idx: [     3     22     31 ... 149189 149190 149214]
+ train / valid: [     3     22     31 ... 149189 149190 149214]
Fold 2: 
+ train_idx: [     0      2      3 ... 149215 149216 149217]
+ valid_idx: [     1      6      7 ... 149203 149205 149208]
+ train / valid: [     1      6      7 ... 149203 149205 149208]
Fold 3: 
+ train_idx: [     0      1      3 ... 149214 149215 149217]
+ valid_idx: [     2      9     10 ... 149197 149207 149216]
+ train / valid: [     2      9     10 ... 149197 149207 149216]
Fold 4: 
+ train_idx: [     0      1      2 ... 149215 149216 149217]
+ valid_idx: [     5     15     16 ... 149200 149209 149213]
+ train / valid: [     5     15     16 ... 149200 149209 149213]


In [24]:
# for column in outliers_list:
#     data_test = remove_outliers_iqr(data_test, column)
# data_test['collision_type'].replace('?',np.nan,inplace=True)
# data_test['collision_type'].fillna(method='ffill',inplace=True)
# data_test[['property_damage','police_report_available']].replace('?','Unknow',inplace=True)
# (data_test=='?').sum()
scaled_df = scaleColumns(test_con,['PERFORM_CNS_SCORE','PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 
                                 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_OVERDUE_ACCTS', 
                                 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT',
                                 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 
                                 'NO_OF_INQUIRIES', 'DISBURSED_AMOUNT_new',
                                 'ASSET_COST_new', 'LTV_new', 'age'])

test_dummy = pd.get_dummies(scaled_df, prefix_sep='_', drop_first=True)
y_test = test_dummy[['LOAN_DEFAULT']]
x_test= test_dummy.loc[:, test_dummy.columns != 'LOAN_DEFAULT']
x_test.to_csv(f'{save_dir}/x_test.csv', index=False)
y_test.to_csv(f'{save_dir}/y_test.csv', index=False)